In [1]:
import pandas as pd 
import sklearn.metrics
from sklearn.metrics import confusion_matrix
import numpy as np
import seaborn as sns
import scipy.stats as stats
import pandas as pd
import matplotlib.pyplot as plt
from pydataset import data
import env
import acquire as acq
import prepare as prp
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier,export_text,plot_tree
from sklearn.metrics import accuracy_score, classification_report

from sklearn.ensemble import RandomForestClassifier

In [13]:
# load titanic via acquire.py
df = acq.get_titanic_data('titanic_db')
df = prp.prep_titanic(df)
df.head()
# Split dataset
train, validate, test = prp.split_data(df,'survived')
train.shape,validate.shape,test.shape
#Determine drivers of target
train.columns[:-2]
cat_cols, num_cols = [], []
for col in train.columns[:-2]:
    if train[col].dtype == 'O':
        cat_cols.append(col)
    else:
        if train[col].nunique() < 10:
            cat_cols.append(col)
        else:
            num_cols.append(col)
cat_cols,num_cols
explore_cols = cat_cols + num_cols
explore_cols
train.survived.value_counts()
train['baseline_pred'] = 0
train.head(10)
pd.crosstab(train.baseline_pred,train.survived) 
# Baseline ACCURACY is 61.6%  
baseline_acc = (train.baseline_pred == train.survived).mean()
baseline_acc
X_cols = train.columns.to_list()
X_cols.remove('survived')
X_cols.remove('baseline_pred')
X_cols.remove('sex')
X_cols.remove('embark_town')
y_cols = 'survived'
X_cols

['pclass',
 'age',
 'sibsp',
 'parch',
 'fare',
 'alone',
 'sex_male',
 'embark_town_Queenstown',
 'embark_town_Southampton']

In [28]:
# make the thing
rf1 = RandomForestClassifier(random_state=2013,max_depth=10)

In [29]:
X_train = train[X_cols]
y_train = train[y_cols]

In [30]:
# fit the thing
# note: if we narrow down the feature in fit:
rf1.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, random_state=2013)

In [31]:
# use the thing
# we need to do it on predict as well!!
rf1_preds = rf1.predict(X_train)

In [32]:
# narrow down independent and dependent features with my
# lists of features from before
X_val, y_val = validate[X_cols], validate[y_cols]

In [33]:
print(f'''
Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): {rf1.score(X_train, y_train)}
Accuracy (validate): {round(rf1.score(X_val, y_val), 2)}''')


Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): 0.9618473895582329
Accuracy (validate): 0.81


In [39]:
# RandForest classification report:
pd.DataFrame(classification_report(y_train, rf1_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.947205,0.988636,0.961847,0.967921,0.963095
recall,0.993485,0.910995,0.961847,0.952240,0.961847
f1-score,0.969793,0.948229,0.961847,0.959011,0.961523
support,307.000000,191.000000,0.961847,498.000000,498.000000


In [42]:
pd.crosstab(rf1_preds,train[y_cols])

TP = 305
FP = 17
FN = 2
TN = 174
pd.crosstab(rf1_preds,train[y_cols])

survived,0,1
row_0,,
0,305,17
1,2,174


In [ ]:
# Not needed yet, but just want to see the dropoff when looking at validate...

In [35]:
validate_pred = rf1.predict(X_val)

In [36]:
# RandForest classification report:
pd.DataFrame(classification_report(y_val, validate_pred, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.837037,0.759494,0.808411,0.798265,0.807324
recall,0.856061,0.731707,0.808411,0.793884,0.808411
f1-score,0.846442,0.745342,0.808411,0.795892,0.807703
support,132.000000,82.000000,0.808411,214.000000,214.000000


In [40]:
pd.crosstab(validate_pred,validate[y_cols])

survived,0,1
row_0,,
0,113,22
1,19,60


In [43]:
## ok, back to TRAIN data...
pd.crosstab(rf1_preds,train[y_cols])

survived,0,1
row_0,,
0,305,17
1,2,174


In [45]:
pd.DataFrame(classification_report(y_train, rf1_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.947205,0.988636,0.961847,0.967921,0.963095
recall,0.993485,0.910995,0.961847,0.952240,0.961847
f1-score,0.969793,0.948229,0.961847,0.959011,0.961523
support,307.000000,191.000000,0.961847,498.000000,498.000000


####  TRAIN ACCURACY-TPR(recall for 1)-FPR()-TNR()-FNR()-PRECISION (for 1)-RECALL (for 1)-F1, Support

In [44]:
sklearn.metrics.accuracy_score(train[y_cols],rf1_preds)

0.9618473895582329

In [48]:
# TPR == Recall
sklearn.metrics.recall_score(train[y_cols],rf1_preds)

0.9109947643979057

In [49]:
# FalsePosRate FP / (FP + TN) ...also, this is  1 - Recall for case of 'survived'

FP / (FP+TN)

0.08900523560209424

In [50]:
# TrueNegRate
1 - (FP / (FP+TN))

0.9109947643979057

In [52]:
# FalseNegRate FN / (TP+FP)  1- recall

1 - (sklearn.metrics.recall_score(train[y_cols],rf1_preds))

0.08900523560209428

In [53]:
sklearn.metrics.precision_score(train[y_cols],rf1_preds)

0.9886363636363636

In [54]:
sklearn.metrics.recall_score(train[y_cols],rf1_preds)

0.9109947643979057

In [55]:
sklearn.metrics.f1_score(train[y_cols],rf1_preds)

0.9482288828337875

In [56]:
print(
    classification_report(train[y_cols],
                      rf1_preds))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       307
           1       0.99      0.91      0.95       191

    accuracy                           0.96       498
   macro avg       0.97      0.95      0.96       498
weighted avg       0.96      0.96      0.96       498



In [57]:
# make the thing
rf2 = RandomForestClassifier(random_state=2013,max_depth=10,min_samples_leaf=5)
X_train = train[X_cols]
y_train = train[y_cols]
# fit the thing
# note: if we narrow down the feature in fit:
rf2.fit(X_train, y_train)
# use the thing
# we need to do it on predict as well!!
rf2_preds = rf2.predict(X_train)
# lists of features from before
X_val, y_val = validate[X_cols], validate[y_cols]
print(f'''
Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): {rf2.score(X_train, y_train)}
Accuracy (validate): {round(rf2.score(X_val, y_val), 2)}''')


Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): 0.8855421686746988
Accuracy (validate): 0.79


In [61]:
# pd.crosstab(rf1_preds,train[y_cols])
# RandForest classification report:
pd.DataFrame(classification_report(y_train, rf2_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.872024,0.913580,0.885542,0.892802,0.887962
recall,0.954397,0.774869,0.885542,0.864633,0.885542
f1-score,0.911353,0.838527,0.885542,0.874940,0.883422
support,307.000000,191.000000,0.885542,498.000000,498.000000


In [62]:
# make the thing
rf3 = RandomForestClassifier(random_state=2013,max_depth=7,min_samples_leaf=11)
X_train = train[X_cols]
y_train = train[y_cols]
# fit the thing
# note: if we narrow down the feature in fit:
rf3.fit(X_train, y_train)
# use the thing
# we need to do it on predict as well!!
rf3_preds = rf3.predict(X_train)
# lists of features from before
X_val, y_val = validate[X_cols], validate[y_cols]
print(f'''
Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): {rf3.score(X_train, y_train)}
Accuracy (validate): {round(rf3.score(X_val, y_val), 2)}''')


Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): 0.8393574297188755
Accuracy (validate): 0.79


In [63]:
# pd.crosstab(rf1_preds,train[y_cols])
# RandForest classification report:
pd.DataFrame(classification_report(y_train, rf3_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.821530,0.882759,0.839357,0.852144,0.845013
recall,0.944625,0.670157,0.839357,0.807391,0.839357
f1-score,0.878788,0.761905,0.839357,0.820346,0.833959
support,307.000000,191.000000,0.839357,498.000000,498.000000


In [64]:
# make the thing
rf4 = RandomForestClassifier(random_state=2013,max_depth=5,min_samples_leaf=25)
X_train = train[X_cols]
y_train = train[y_cols]
# fit the thing
# note: if we narrow down the feature in fit:
rf4.fit(X_train, y_train)
# use the thing
# we need to do it on predict as well!!
rf4_preds = rf4.predict(X_train)
# lists of features from before
X_val, y_val = validate[X_cols], validate[y_cols]
print(f'''
Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): {rf4.score(X_train, y_train)}
Accuracy (validate): {round(rf4.score(X_val, y_val), 2)}''')


Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): 0.8172690763052208
Accuracy (validate): 0.79


In [65]:
# pd.crosstab(rf1_preds,train[y_cols])
# RandForest classification report:
pd.DataFrame(classification_report(y_train, rf4_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.790323,0.896825,0.817269,0.843574,0.831170
recall,0.957655,0.591623,0.817269,0.774639,0.817269
f1-score,0.865979,0.712934,0.817269,0.789457,0.807281
support,307.000000,191.000000,0.817269,498.000000,498.000000


In [66]:
# make the thing
rf5 = RandomForestClassifier(random_state=2013,max_depth=2,min_samples_leaf=45)
X_train = train[X_cols]
y_train = train[y_cols]
# fit the thing
# note: if we narrow down the feature in fit:
rf5.fit(X_train, y_train)
# use the thing
# we need to do it on predict as well!!
rf5_preds = rf5.predict(X_train)
# lists of features from before
X_val, y_val = validate[X_cols], validate[y_cols]
print(f'''
Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): {rf5.score(X_train, y_train)}
Accuracy (validate): {round(rf5.score(X_val, y_val), 2)}''')


### POSSIBLE UNDERFIT?


Performance in accuracy of Random Forest 1 on training data:
Accuracy (train): 0.7751004016064257
Accuracy (validate): 0.77


In [67]:
# pd.crosstab(rf1_preds,train[y_cols])
# RandForest classification report:
pd.DataFrame(classification_report(y_train, rf5_preds, output_dict=True))

,0,1,accuracy,macro avg,weighted avg
precision,0.764228,0.806202,0.7751,0.785215,0.780326
recall,0.918567,0.544503,0.7751,0.731535,0.775100
f1-score,0.834320,0.650000,0.7751,0.742160,0.763627
support,307.000000,191.000000,0.7751,498.000000,498.000000


RANDOPM FOREST 1 perform s best on the TRAIN set at 96.1% because it is not limited by min_samples per leaf or a low max_depth number...however it shows the gretaes dropoff when compared to validation numbers...it is severly OVERFIT